In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import optimizers
import keras_cv
import numpy as np
import pandas as pd

from keras_cv import bounding_box
import os
import resource
from keras_cv import visualization
import tqdm

import matplotlib.pyplot as plt
import cv2

from sklearn.metrics import auc
import time
import pickle
import scipy.interpolate


In [ ]:
print(keras_cv.__version__)

In [ ]:
BATCH_SIZE = 16
EPOCHS = 200
CHECKPOINT_PATH = os.getenv("CHECKPOINT_PATH", "checkpoint/")
INFERENCE_CHECKPOINT_PATH = os.getenv("INFERENCE_CHECKPOINT_PATH", CHECKPOINT_PATH)

low, high = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (high, high))

In [ ]:
class_label_dic = {0:"antralfollicle",
                   1:"antralfolliclewn",
                   2:"corpusluteum",
                   #3:"negative"
                  }

img_width = 640
img_height = 640

"""img width and size depend on the backbone architecture, for example to use the RetinaNet architecture 
with a ResNet50 backbone, we need to resize our image to a size that is divisible by 64. This is to ensure 
compatibility with the number of downscaling operations done by
the convolution layers in the ResNet."""


In [ ]:
os.path.abspath(os.getcwd())

In [ ]:
with tf.device("CPU"):
    with open('tf_data_samp/train' + '/element_spec', 'rb') as in_:
        elemspec = pickle.load(in_)
    train_ds = tf.data.experimental.load('tf_data_samp/train', elemspec, compression='GZIP')
    
    with open('tf_data_samp/val' + '/element_spec', 'rb') as in_:
        elemspec = pickle.load(in_)
    eval_ds = tf.data.experimental.load('tf_data_samp/val', elemspec, compression='GZIP')
    
    with open('tf_data_samp/test' + '/element_spec', 'rb') as in_:
        elemspec = pickle.load(in_)
    test_ds = tf.data.experimental.load('tf_data_samp/test', elemspec, compression='GZIP')

In [ ]:
#To convert boundary boxes to ragged tensor format

def dict_to_tuple(inputs):
    return {"images": tf.RaggedTensor.from_tensor(inputs["images"]), 
            "bounding_boxes": bounding_box.to_ragged(inputs["bounding_boxes"])}
with tf.device("CPU"):
    train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    eval_ds = eval_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Here we are batching our dataset.
with tf.device("CPU"):
    train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
    eval_ds = eval_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
    test_ds = test_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
#Here we use the kerasCV function to visualize our dataset

def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_label_dic,
    )

with tf.device("CPU"):
    visualize_dataset(train_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)

In [ ]:
"""The most demanding task data augmentation is being done here.
KerasCV supports bounding box augmentation with its library of data augmentation layers.
"""
with tf.device("CPU"):
    augmenter = keras.Sequential(
        layers=[
            keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xywh"),
            #keras_cv.layers.preprocessing.Grayscale(),
            keras_cv.layers.JitteredResize(target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xywh"),
            #keras_cv.layers.RandomShear(x_factor=(0.2, 0.2), y_factor=(0.2, 0.2),bounding_box_format="xywh"),
            #keras_cv.layers.Mosaic(bounding_box_format="xywh")
            #keras_cv.layers.MixUp() 
        ]
    )
    
    train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)
    visualize_dataset(train_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)


In [ ]:
with tf.device("CPU"):
    visualize_dataset(eval_ds,bounding_box_format="xywh",value_range=(0, 255),rows=2,cols=3,# path="eval.png"
                     )


In [ ]:
#Gray scale images
res_gray = keras.Sequential(
        layers=[keras_cv.layers.Resizing(640, 640, bounding_box_format="xywh", pad_to_aspect_ratio=True),
                keras_cv.layers.preprocessing.Grayscale(output_channels=3),
               ])
train_ds = train_ds.map(res_gray, num_parallel_calls=tf.data.AUTOTUNE)
visualize_dataset(train_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)
eval_ds = eval_ds.map(res_gray, num_parallel_calls=tf.data.AUTOTUNE)
visualize_dataset(eval_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)
test_ds = test_ds.map(res_gray, num_parallel_calls=tf.data.AUTOTUNE)
visualize_dataset(test_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)

In [ ]:

with tf.device("CPU"):

    inference_resizing = keras_cv.layers.Resizing(640, 640, bounding_box_format="xywh", pad_to_aspect_ratio=True)
    train_ds = train_ds.map(inference_resizing, num_parallel_calls=tf.data.AUTOTUNE)

    visualize_dataset(train_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)

In [ ]:

with tf.device("CPU"):

    inference_resizing = keras_cv.layers.Resizing(640, 640, bounding_box_format="xywh", pad_to_aspect_ratio=True)
    eval_ds = eval_ds.map(inference_resizing, num_parallel_calls=tf.data.AUTOTUNE)

    visualize_dataset(eval_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)

In [ ]:

with tf.device("CPU"):

    inference_resizing = keras_cv.layers.Resizing(640, 640, bounding_box_format="xywh", pad_to_aspect_ratio=True)
    test_ds = test_ds.map(inference_resizing, num_parallel_calls=tf.data.AUTOTUNE)

    visualize_dataset(test_ds, bounding_box_format="xywh", value_range=(0, 255), rows=2, cols=3)


In [ ]:
"""
Here we prepare the data to feed into our model by upacking from preprocessing dictionary.
We need to unpack only in case of ragged tensors which needs to be converted to to dense tensors.
"""

def dict_to_tuple(inputs):
    return inputs["images"], bounding_box.to_dense(inputs["bounding_boxes"], max_boxes=32)

with tf.device("CPU"):
    train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    eval_ds = eval_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    test_ds = test_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)



In [ ]:
"""the most important call is prefetch: fetch the data to create 
data in the background while current data is being processed"""
with tf.device("CPU"):
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    eval_ds = eval_ds.prefetch(tf.data.AUTOTUNE)
    #test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
coco_metrics = keras_cv.metrics.BoxCOCOMetrics(bounding_box_format="xywh", evaluate_freq=1)

In [ ]:
def print_metrics(metrics):
    maxlen = max([len(key) for key in result.keys()])
    print("Metrics:")
    print("-" * (maxlen + 1))
    for k, v in metrics.items():
        print(f"{k.ljust(maxlen+1)}: {v.numpy():0.6f}")
        

In [ ]:
# Visualizing predicted boxes

with tf.device("CPU"):
    visualization_ds = eval_ds.unbatch()
    visualization_ds = visualization_ds.ragged_batch(16)
    visualization_ds = visualization_ds.shuffle(8)

class VisualizeDetections(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        visualize_detections(self.model, bounding_box_format="xywh", dataset=visualization_ds)



In [ ]:
def visualize_detections(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
    y_pred = model.predict(images)
    y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=4,
        rows=2,
        cols=4,
        show=True,
        font_scale=0.7,
        class_mapping=class_label_dic,
    )


In [ ]:
base_lr = 0.005
# including a global_clipnorm is extremely important in object detection tasks
optimizer = tf.keras.optimizers.SGD(
    learning_rate=base_lr, momentum=0.9, global_clipnorm=10.0
)


In [ ]:
model = keras_cv.models.RetinaNet.from_preset(
    "mobilenet_v3_large_imagenet",
    num_classes=len(class_label_dic),
    bounding_box_format="xywh",
)


In [ ]:


model_checkpoint_callback = [
    keras.callbacks.ReduceLROnPlateau(patience=10),
    keras.callbacks.EarlyStopping(patience=20),
    keras.callbacks.TensorBoard(log_dir="logs"),
    keras.callbacks.ModelCheckpoint(os.path.join(CHECKPOINT_PATH, 'model.{epoch:02d}-{val_MaP:.2f}'), 
                                    save_best_only=True, mode = 'max',monitor='val_MaP',
                                    save_weights_only=True)
]


In [ ]:
model.compile(
    classification_loss="focal", 
    box_loss="smoothl1",
    optimizer=optimizer,
    metrics=[coco_metrics],
)


In [ ]:
start_time = time.time()

history = model.fit(
    train_ds,
    validation_data= eval_ds,
    epochs=EPOCHS,
    callbacks=[VisualizeDetections(), model_checkpoint_callback],
)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
path = 'train_history/mobilenet_v3_large_imagenet/'
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
with open(os.path.join(path,'history_wp'), 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
path = 'train_history/mobilenet_v3_large_imagenet/'
with open(os.path.join(path,'history_wp'), "rb") as file_pi:
    history = pickle.load(file_pi)

In [ ]:
model.summary()

In [ ]:
#Plot MaP curves

plt.plot(history['MaP'])
plt.plot(history['val_MaP'])
plt.title('Mean avg Precision')
plt.ylabel('MaP')
plt.xlabel('epoch')
plt.legend(['MaP', 'val_MaP'], loc='upper right')
plt.show()


In [ ]:
#Plot accuracy and loss curves


# list all data in history
print(history.keys())
# summarize history for loss
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper right')
plt.show()



In [ ]:
# list all data in history
print(history.keys())
# summarize history for loss
plt.plot(history['box_loss'])
plt.plot(history['val_box_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['box_loss', 'val_box_loss'], loc='upper right')
plt.show()


In [ ]:
# list all data in history
print(history.keys())
# summarize history for loss
plt.plot(history['classification_loss'])
plt.plot(history['val_classification_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['classification_loss', 'val_classification_loss'], loc='upper right')
plt.show()

In [ ]:
# list all data in history
print(history.keys())
# summarize history for loss
plt.plot(history['percent_boxes_matched_with_anchor'])
plt.plot(history['val_percent_boxes_matched_with_anchor'])
plt.title('percent_boxes_matched_with_anchor')
plt.ylabel('percent_boxes_matched_with_anchor')
plt.xlabel('epoch')
plt.legend(['percent_boxes_matched_with_anchor', 'val_percent_boxes_matched_with_anchor'])
plt.show()

In [ ]:
model.load_weights('train_history/mobilenet_v3_large_imagenet/with_sampling/no_aug/checkpoint/model.47-0.46')

In [ ]:

#Keras CV evaluation metrics

#Ragged tensor needs to be converted to the dense tensor in order to work with COCO metric, 

coco_metrics = keras_cv.metrics.BoxCOCOMetrics(bounding_box_format="xywh", evaluate_freq=1)

model.compile(
    classification_loss="focal",
    box_loss="smoothl1",
    optimizer=optimizer,
    metrics=[coco_metrics],
    )

coco_metrics.reset_state()
result = model.evaluate(eval_ds.take(-1), verbose=0)
result = coco_metrics.result(force=True)

print_metrics(result)

"""The most common metric to evaluate performance of Object detection model is Mean average
Precision (MaP).
Average precision calculates area under the precision-recall curve.
We want to increase the value of MaP.
"""

In [ ]:
# Visualizing predicted boxes

visualization_ds = test_ds.unbatch()
visualization_ds = visualization_ds.ragged_batch(BATCH_SIZE)
#visualization_ds = visualization_ds.shuffle(8)

In [ ]:
"""Data is divided into different batches of size defined in the beginning"""
coco_metrics = keras_cv.metrics.BoxCOCOMetrics(bounding_box_format="xywh", evaluate_freq=1)

eval_ub = test_ds.unbatch()
eval_ub = eval_ub.ragged_batch(len(eval_ub))

coco_metrics.reset_state()
y_p = []
y_t = []
for images, y_true in tqdm.tqdm(iter(eval_ub)):
    y_pred = model.predict(images, verbose=0)
    coco_metrics.update_state(y_true, y_pred)
    y_p.append(y_pred)
    y_t.append(bounding_box.to_ragged(y_true))
    print_metrics(coco_metrics.result(force=True))
result = coco_metrics.result(force=True)
print_metrics(result)

In [ ]:

model.prediction_decoder = keras_cv.layers.MultiClassNonMaxSuppression(
    bounding_box_format="xywh",
    from_logits=True,
    iou_threshold=0.75,
    confidence_threshold=0.75,
)


eval_ub = eval_ds.unbatch()
eval_ub = eval_ub.ragged_batch(8)

i = 0
for images, y_true in tqdm.tqdm(iter(eval_ub)):
    i = i+1
    y_pred = model.predict(images)
    y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
            images,
            value_range=(0, 255),
            bounding_box_format="xywh",
            y_true=y_true,
            y_pred=y_pred,
            scale=8,
            rows=2,
            cols=3,
            #show=True,
            font_scale=0.6,
            class_mapping=class_label_dic,
            legend=True,
            path=os.path.join('eval_images','{}'.format(i)),
    )
       